In [54]:
import pandas as pd
import numpy as np

import struct
import binascii
import ctypes

import time
import socket

In [55]:
# df = pd.read_csv('../data/processed_data/all/resample_to_50_per_act/file_split/test_xyz.csv')
# df = df.iloc[:50]
df = pd.read_csv('../data/X3/zyu/test/allnolabel.csv')
df = df.iloc[13700:13800]
df.head()
df_len = len(df)
df_len

,AccelerometerX,AccelerometerY,AccelerometerZ
13700,1.30244,-9.11711,-3.88818
13701,1.30244,-9.11711,-3.88818
13702,1.16837,-8.81066,-3.75411
13703,1.03429,-8.75319,-3.83072
13704,0.74699,-8.40843,-3.75411


100

In [56]:
i = 0
arr = np.array(df.iloc[i:i+1]).reshape(3)
type(arr[0])
arr
arr = np.around((arr*32768/2/9.8/100)).astype(int)
values = tuple(arr)
type(values[0])
values

numpy.float64

array([ 1.30244, -9.11711, -3.88818])

numpy.int32

(22, -152, -65)

In [57]:
s = struct.Struct('<3h')
packed_data = s.pack(*values)
type(packed_data)
len(packed_data)

bytes

6

In [58]:
sample_points = 100
buffer_size = s.size*sample_points
buffer_size
prebuffer = ctypes.create_string_buffer(buffer_size)
len(binascii.hexlify(prebuffer))

s.pack_into(prebuffer,0,*values)
binascii.hexlify(prebuffer)
len(binascii.hexlify(prebuffer))

s.unpack_from(prebuffer,0)

600

1200

b'160068ffbfff00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

1200

(22, -152, -65)

In [59]:
s = struct.Struct('<3h')
sample_point_size = s.size
sample_points = 100
buffer_size = sample_point_size * sample_points
prebuffer = ctypes.create_string_buffer(buffer_size)

ip_port = ('127.0.0.1',9999)
sk = socket.socket(socket.AF_INET,socket.SOCK_DGRAM,0)

df_idx = 0
buffer_idx = 0

while True:
    if df_idx == df_len:
        break
    time.sleep(0.04)
    arr = np.array(df.iloc[df_idx:df_idx+1]).reshape(3)
    arr = np.around((arr*32768/4/9.8/100)).astype(int)
    values = tuple(arr)
    s.pack_into(prebuffer,buffer_idx,*values)
    df_idx += 1
    buffer_idx += sample_point_size
    if buffer_idx == buffer_size:
        print('发送编码数据：%s' %binascii.hexlify(prebuffer))
        sk.sendto(binascii.hexlify(prebuffer),ip_port)
        buffer_idx = 0

发送编码数据：b'0b00b4ffdfff0b00b4ffdfff0a00b6ffe1ff0900b7ffe0ff0600baffe1ff0300beffe3ff0000c1ffe6fffeffc4ffeafffcffc6ffebfff9ffc9ffedfff7ffccffeefff6ffcffff1fff6ffd3fff5fff7ffd5fff8fff8ffd3fffafffaffcffffafffcffcafff5ff0100c0ffecff0800b6ffe4ff0b00aeffdcff0b00a4ffd4ff10009affd7ff100096ffdbff0e0094ffd9ff0b0090ffd9ff080091ffdcff050093ffdfff020095ffe2ff030098ffe0ff05009bffdfff07009effdfff0700a5ffdfff0700a9ffdfff0800aeffdeff0800b1ffdfff0600b6ffe0ff0300bbffe2ff0000bfffe5fffeffc3ffe8fffdffc7ffeafffaffccffebfffaffd1ffecfffcffd7ffeffffcffdafff1fffcffdafff2fffdffd8fff2fffdffd3fff2ffffffccfff2ff0200c3ffefff0500bdffe9ff0a00afffe3ff10009bffe1ff130092ffe0ff11008effdcff0d008affd5ff0b0087ffd3ff08008bffdaff05008fffdeff020092ffe0ff040096ffe0ff07009bffe1ff0700a1ffe2ff0700a7ffe1ff0900adffe1ff0900b1ffe1ff0800b4ffe1ff0600b9ffe3ff0400bcffe5ff0200c0ffe7ff0000c4ffe8fffeffc8ffebfffbffceffedfffbffd2ffeffffcffd7fff1fffbffd7fff2fffdffd7fff2fffeffd5fff1ffffffd0fff0ff0100c8ffefff0300bfffebff0700b6ffe9ff0900adffe8ff0b00a2f

1200

In [60]:
s = struct.Struct('<300h')
rcvs = np.array(s.unpack_from(prebuffer,0))
rcvs
len(rcvs)

array([  11,  -76,  -33,   11,  -76,  -33,   10,  -74,  -31,    9,  -73,
        -32,    6,  -70,  -31,    3,  -66,  -29,    0,  -63,  -26,   -2,
        -60,  -22,   -4,  -58,  -21,   -7,  -55,  -19,   -9,  -52,  -18,
        -10,  -49,  -15,  -10,  -45,  -11,   -9,  -43,   -8,   -8,  -45,
         -6,   -6,  -49,   -6,   -4,  -54,  -11,    1,  -64,  -20,    8,
        -74,  -28,   11,  -82,  -36,   11,  -92,  -44,   16, -102,  -41,
         16, -106,  -37,   14, -108,  -39,   11, -112,  -39,    8, -111,
        -36,    5, -109,  -33,    2, -107,  -30,    3, -104,  -32,    5,
       -101,  -33,    7,  -98,  -33,    7,  -91,  -33,    7,  -87,  -33,
          8,  -82,  -34,    8,  -79,  -33,    6,  -74,  -32,    3,  -69,
        -30,    0,  -65,  -27,   -2,  -61,  -24,   -3,  -57,  -22,   -6,
        -52,  -21,   -6,  -47,  -20,   -4,  -41,  -17,   -4,  -38,  -15,
         -4,  -38,  -14,   -3,  -40,  -14,   -3,  -45,  -14,   -1,  -52,
        -14,    2,  -61,  -17,    5,  -67,  -23,   

300

In [61]:
rcvs = rcvs/32768*4*9.8*100

In [62]:
x = rcvs[0:300:3]
y = rcvs[1:300:3]
z = rcvs[2:300:3]
x
y
z

array([ 1.31591797,  1.31591797,  1.19628906,  1.07666016,  0.71777344,
        0.35888672,  0.        , -0.23925781, -0.47851563, -0.83740234,
       -1.07666016, -1.19628906, -1.19628906, -1.07666016, -0.95703125,
       -0.71777344, -0.47851563,  0.11962891,  0.95703125,  1.31591797,
        1.31591797,  1.9140625 ,  1.9140625 ,  1.67480469,  1.31591797,
        0.95703125,  0.59814453,  0.23925781,  0.35888672,  0.59814453,
        0.83740234,  0.83740234,  0.83740234,  0.95703125,  0.95703125,
        0.71777344,  0.35888672,  0.        , -0.23925781, -0.35888672,
       -0.71777344, -0.71777344, -0.47851563, -0.47851563, -0.47851563,
       -0.35888672, -0.35888672, -0.11962891,  0.23925781,  0.59814453,
        1.19628906,  1.9140625 ,  2.27294922,  2.03369141,  1.55517578,
        1.31591797,  0.95703125,  0.59814453,  0.23925781,  0.47851563,
        0.83740234,  0.83740234,  0.83740234,  1.07666016,  1.07666016,
        0.95703125,  0.71777344,  0.47851563,  0.23925781,  0.  

array([ -9.09179688,  -9.09179688,  -8.85253906,  -8.73291016,
        -8.37402344,  -7.89550781,  -7.53662109,  -7.17773438,
        -6.93847656,  -6.57958984,  -6.22070312,  -5.86181641,
        -5.38330078,  -5.14404297,  -5.38330078,  -5.86181641,
        -6.45996094,  -7.65625   ,  -8.85253906,  -9.80957031,
       -11.00585938, -12.20214844, -12.68066406, -12.91992188,
       -13.3984375 , -13.27880859, -13.03955078, -12.80029297,
       -12.44140625, -12.08251953, -11.72363281, -10.88623047,
       -10.40771484,  -9.80957031,  -9.45068359,  -8.85253906,
        -8.25439453,  -7.77587891,  -7.29736328,  -6.81884766,
        -6.22070312,  -5.62255859,  -4.90478516,  -4.54589844,
        -4.54589844,  -4.78515625,  -5.38330078,  -6.22070312,
        -7.29736328,  -8.01513672,  -9.68994141, -12.08251953,
       -13.15917969, -13.63769531, -14.11621094, -14.47509766,
       -13.99658203, -13.51806641, -13.15917969, -12.68066406,
       -12.08251953, -11.36474609, -10.64697266,  -9.92

array([-3.94775391, -3.94775391, -3.70849609, -3.828125  , -3.70849609,
       -3.46923828, -3.11035156, -2.63183594, -2.51220703, -2.27294922,
       -2.15332031, -1.79443359, -1.31591797, -0.95703125, -0.71777344,
       -0.71777344, -1.31591797, -2.39257812, -3.34960938, -4.30664062,
       -5.26367188, -4.90478516, -4.42626953, -4.66552734, -4.66552734,
       -4.30664062, -3.94775391, -3.58886719, -3.828125  , -3.94775391,
       -3.94775391, -3.94775391, -3.94775391, -4.06738281, -3.94775391,
       -3.828125  , -3.58886719, -3.22998047, -2.87109375, -2.63183594,
       -2.51220703, -2.39257812, -2.03369141, -1.79443359, -1.67480469,
       -1.67480469, -1.67480469, -1.67480469, -2.03369141, -2.75146484,
       -3.46923828, -3.70849609, -3.828125  , -4.30664062, -5.14404297,
       -5.38330078, -4.54589844, -4.06738281, -3.828125  , -3.828125  ,
       -3.70849609, -3.58886719, -3.70849609, -3.70849609, -3.70849609,
       -3.70849609, -3.46923828, -3.22998047, -2.99072266, -2.87

In [63]:
from sklearn.externals import joblib
scaler = joblib.load('../data/X3/zyu/fm/scaler_train.pkl')

In [64]:
import feature_project as fp
X = fp.feature_extraction(x,y,z).reshape(1, -1).astype(float)

In [65]:
X = scaler.transform(X)

In [66]:
X

array([[ -4.98899358e-01,  -1.44897493e+00,  -1.43319288e+00,
         -4.98899358e-01,  -1.44897493e+00,  -1.43319288e+00,
         -7.09897956e-01,  -3.62497868e-03,  -5.05196955e-01,
          2.26955886e-01,   3.61192304e-02,   1.66611590e-02,
         -6.81922599e-01,  -1.09102092e-01,  -4.39825543e-01,
         -6.90085028e-01,  -3.65680662e-01,  -9.72328204e-01,
         -3.82128694e-01,  -1.62182468e+00,  -1.48401738e+00,
         -3.44511033e-01,  -1.01125215e+00,  -1.30559033e+00,
         -5.77795940e-01,  -1.31532698e-01,  -8.56524558e-01,
         -4.37413872e-01,  -1.43353964e+00,  -1.34992600e+00,
         -3.03866966e-01,   1.18413699e+00,   8.25518291e-01,
          4.41601994e-01,   7.24522633e-01,   5.76321640e-01,
         -4.00180206e-01,  -1.43382272e-01,   5.98239837e-01,
         -3.43491401e-01,  -3.42643472e-01,  -3.59528318e-01,
         -1.71854250e+00,   6.59379429e-01,  -3.99010883e-01,
         -4.37399049e+00,   1.70683285e+00,   4.43524961e-01,
        

In [67]:
clf = joblib.load('./modelX3/svm_clf.pkl')
predict_label = clf.predict(X)[0]
predict_label

'dump'